In [1]:
res = readRDS("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/deletion.genes.block30.binomial.whole.genome.prior.varbvs.nologodds.rds")
# res1 = readRDS("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/deletion.genes.block30.binomial.whole.genome.prior.varbvs.nologodds.rds")

In [12]:
res$logodds[17]

[1] -1.528852

In [10]:
res$sa[17]

[1] 1.903851

In [17]:
dim(res$eta)

[1] 13412    20

In [2]:
dim(res$alpha); dim(res$mu); dim(res$s)

[1] 22356    20

[1] 22356    20

[1] 22356    20

In [15]:
varbvs::varbvs

function (X, Z, y, family = c("gaussian", "binomial"), sigma, 
    sa, logodds, weights, resid.vcov, alpha, mu, eta, update.sigma, 
    update.sa, optimize.eta, initialize.params, update.order, 
    nr = 100, sa0 = 1, n0 = 10, tol = 1e-04, maxiter = 10000, 
    verbose = TRUE) 
{
    n <- nrow(X)
    p <- ncol(X)
    if (!(is.matrix(X) & is.numeric(X) & sum(is.na(X)) == 0)) 
        stop("Input X must be a numeric matrix with no missing values.")
    storage.mode(X) <- "double"
    if (!is.null(Z)) {
        Z <- as.matrix(Z)
        if (!is.numeric(Z) | sum(is.na(Z)) > 0) 
            stop("Input Z must be a numeric matrix with no missing values.")
        if (nrow(Z) != n) 
            stop("Inputs X and Z do not match.")
        storage.mode(Z) <- "double"
    }
    if (is.null(Z)) 
        Z <- matrix(1, n, 1)
    else Z <- cbind(1, Z)
    if (!is.numeric(y) | sum(is.na(y)) > 0) 
        stop("Input y must be a numeric vector with no missing values.")
    if (length(y) != n) 
        stop("Inputs X and y do not match")
    y <- c(as.double(y))
    family <- match.arg(family)
    if (!is.finite(maxiter)) 
        stop("Input maxiter must be a finite number")
    if (missing(sigma)) {
        sigma <- var(y)
        update.sigma.default <- TRUE
    }
    else {
        sigma <- c(sigma)
        update.sigma.default <- FALSE
        if (family == "binomial") 
            stop("Input sigma is not allowed for family = binomial")
    }
    if (missing(sa)) {
        sa <- 1
        update.sa.default <- TRUE
    }
    else {
        sa <- c(sa)
        update.sa.default <- FALSE
    }
    if (missing(logodds)) {
        if (length(sigma) == 1 & length(sa) == 1) 
            logodds <- seq(-log10(p), -1, length.out = 20)
        else stop("logodds can only be missing when length(sigma) = length(sa) = 1")
    }
    if (!is.matrix(logodds)) {
        prior.same <- TRUE
        logodds <- t(matrix(logodds))
    }
    else if (nrow(logodds) != p) {
        prior.same <- TRUE
        logodds <- t(matrix(logodds))
    }
    else prior.same <- FALSE
    ns <- max(length(sigma), length(sa), ncol(logodds))
    if (length(sigma) == 1) 
        sigma <- rep(sigma, ns)
    if (length(sa) == 1) 
        sa <- rep(sa, ns)
    if (ncol(logodds) == 1) 
        logodds <- rep.col(logodds, ns)
    if (length(sigma) != ns | length(sa) != ns | ncol(logodds) != 
        ns) 
        stop("options.sigma, options.sa and options.logodds are inconsistent")
    if (missing(update.sigma)) 
        update.sigma <- update.sigma.default
    if (missing(update.sa)) 
        update.sa <- update.sa.default
    if (missing(weights)) 
        weights <- NULL
    if (missing(resid.vcov)) 
        resid.vcov <- NULL
    if (!(is.null(weights) & is.null(resid.vcov)) & family != 
        "gaussian") 
        stop(paste("Specifying weights or resid.vcov is only allowed for", 
            "family == \"gaussian\""))
    if (!is.null(weights) & !is.null(resid.vcov)) 
        stop("Only one of weights and resid.vcov may be specified")
    if (!is.null(weights)) 
        if (!(is.vector(weights) & length(weights) == n)) 
            stop("Input weights should be a vector with the same length as y")
    if (!is.null(resid.vcov)) 
        if (is.matrix(resid.vcov) | inherits(resid.vcov, "Matrix")) {
            if (!(nrow(resid.vcov) == n & ncol(resid.vcov) == 
                n & all(resid.vcov == t(resid.vcov)))) 
                stop(paste("Input resid.vcov should be an n x n symmetric", 
                  "matrix where n = length(y)"))
        }
        else stop("Input resid.vcov should be class \"matrix\" or \"Matrix\"")
    initialize.params.default <- TRUE
    if (missing(alpha)) {
        alpha <- rand(p, ns)
        alpha <- alpha/rep.row(colSums(alpha), p)
    }
    else initialize.params.default <- FALSE
    if (!is.matrix(alpha)) 
        alpha <- matrix(alpha)
    if (nrow(alpha) != p) 
        stop("Input alpha must have as many rows as X has columns")
    if (n

In [17]:
pip1 = c(1 - sum((res$alpha %*% res$w)[1216:1274]), (res$alpha %*% res$w)[1216:1274])

In [20]:
mu1 = c(0, res$beta[1216:1274])

In [21]:
sigma.guess1 = sd(res$beta)

In [22]:
sigma.guess1

[1] 0.0603736

In [1]:
library(data.table)
library(reticulate)

In [2]:
library(susieR)
library(BoomSpikeSlab)

Loading required package: Boom
Loading required package: MASS

Attaching package: ‘Boom’

The following object is masked from ‘package:stats’:

    rWishart


Attaching package: ‘BoomSpikeSlab’

The following object is masked from ‘package:stats’:

    knots



In [3]:
X = as.matrix(data.table::fread("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/block_79_137/deletion.genes.block30.for_simu.sample.genes.block_79_137.gz", header = F))

In [4]:
dim(X)

[1] 13412    59

In [5]:
X = cbind(1, X)

In [6]:
y <- as.matrix(data.table::fread("/data/GIT/cnv-gene-mapping/data/deletion_simu/deletion.genes.block30.for_simu.sample.y"))

In [7]:
storage.mode(X) = 'double'
storage.mode(y) = 'double'

In [8]:
prior <- LogitZellnerPrior(X, y, expected.model.size = 1, diagonal.shrinkage = 0, optional.coefficient.estimate = rep(0, dim(X)[2]))

In [9]:
names(prior)

[1] "prior.inclusion.probabilities" "mu"                           
[3] "sigma.guess"                   "prior.df"                     
[5] "sigma.upper.limit"             "max.flips"                    
[7] "siginv"                        "prior.success.probability"

In [10]:
dim(prior$siginv)

[1] 60 60

In [19]:
prior$sigma.guess

[1] 0.7071068

In [23]:
prior$prior.inclusion.probabilities <- pip1
prior$mu <- mu1
prior$sigma.guess <- sigma.guess1

In [24]:
length(prior$mu) == ncol(X)

[1] TRUE

In [25]:
dim(X)

[1] 13412    60

In [62]:
length(prior$mu)

[1] 59

In [ ]:
model <- lm.spike(y ~ X-1, niter = 5000, prior = prior, error.distribution = "gaussian")

In [ ]:
names(model)

In [ ]:
1+1

In [11]:
dim(model$beta)

[1] 1000   59

In [49]:
dim(model$beta)

[1] 1000   59

In [30]:
mean(model$beta[, 20])

[1] 0.5469972

In [13]:
mean(model$beta[, 50])

[1] 0.001787244

In [45]:
res1 = susie(X, y, L = 3, scaled_prior_variance = 0.005, estimate_prior_method = "optim")

In [47]:
max(res1$pip)

[1] 0

In [29]:
res2 = readRDS("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/block_79_137/deletion.genes.block30.for_simu.sample.genes.block_79_137.varbvs.rds")

In [30]:
names(res2)

[1] "fit"  "mu"   "beta"

In [48]:
max(res2$fit$pip)

[1] 0.02698148

**Make input prior as the format of output of SpikeSlabPrior**

pi: vector of prior.inclusion.probabilities

b: optional.coefficient.estimate